In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mobile-price-classification/train.csv
/kaggle/input/mobile-price-classification/test.csv


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
train=pd.read_csv("/kaggle/input/mobile-price-classification/train.csv")
test=pd.read_csv("/kaggle/input/mobile-price-classification/test.csv")

In [4]:
print('Number of NULL values in training data ',sum(train.isna().sum()))
print('Number of NULL values in testing data ',sum(test.isna().sum()))

Number of NULL values in training data  0
Number of NULL values in testing data  0


In [5]:
train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


# Identifying feature reltionship with price

**Battery And Size Analysis**

In [6]:
px.histogram(train,x='battery_power',facet_col='price_range',nbins=10,marginal='box',histnorm='percent')

The low cost phones (price_range=0) is skewed towards the low battery power region and the high cost phones (price_range=3) is skewed towards the higher bettery power region.

In [7]:
px.histogram(train,x='talk_time',facet_col='price_range',nbins=10,marginal='box',histnorm='percent')

The time on singe charge is evenly distributed acorss all price ranges and this quite makes sense, Phones that are having more features will consume the battery much quickly as compared to their counterparts and hence the talk time would be similar.

In [8]:
temp=train
temp['px_active']=(temp['px_height']*temp['px_width'])

In [9]:
px.scatter(temp,x='px_active',color='talk_time',y='battery_power',facet_col='price_range',marginal_x='box')

From above there a few things worth noting,
1. Many higher priced phones have more than 2M active pixels as compared to the other categories which would mean that they either have a large screen size or a better resolution.
2. In the low cost category there are a few phones with less battery life and very high talktime. These could be the older mobile phones with keypad.
3. As intuition suggests a few the higher priced phones have it all, more pixels, large battery size, a longer talktime and of course a higher price.

In [10]:
temp['screen_area']=temp['sc_h']*temp['sc_w']
px.scatter(temp,x='screen_area',color='battery_power',y='px_active',facet_col='price_range',facet_row='touch_screen')

**Camera Analysis**

In [11]:
px.histogram(temp,x='pc',facet_col='price_range',histnorm='percent',nbins=10)

In [12]:
px.histogram(temp,x='fc',facet_col='price_range',histnorm='percent',nbins=10)

Well one thing we can say is high megapixel cameras i.e above 20 MP for primary camera and above 14MP for front camera are very hard to find for any price range.

In [13]:
px.scatter(temp,x='pc',y='fc',color='price_range')

Here's something interesting in the above scatterplot,
1. The phone with the best combination of primary anf front camera falls in the low price region.(Point at the top right corner)
2. The phone with no camera (Point at the botttom left corner) falls in the higher priced category.

In [14]:
print("The low priced phone with the best camera")
print(temp[(temp['fc']==19) & (temp['pc']==20)])
print("The high priced phone with no camera!?")
print(temp[(temp['fc']==0) & (temp['pc']==0) & (temp['price_range']==3)][['pc','fc','price_range','touch_screen']])

The low priced phone with the best camera
      battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
1705           1290     1          1.4         1  19       1          35   

      m_dep  mobile_wt  n_cores  ...  ram  sc_h  sc_w  talk_time  three_g  \
1705    0.3        110        4  ...  879    16     2          8        1   

      touch_screen  wifi  price_range  px_active  screen_area  
1705             0     0            0     300510           32  

[1 rows x 23 columns]
The high priced phone with no camera!?
      pc  fc  price_range  touch_screen
33     0   0            3             1
52     0   0            3             1
67     0   0            3             0
158    0   0            3             0
287    0   0            3             1
524    0   0            3             0
704    0   0            3             0
714    0   0            3             0
763    0   0            3             0
945    0   0            3             1
981    0   0        

In [15]:
print("Number of very high priced phones with no camera ")
print(temp[(train['fc']==0) & (train['pc']==0) & (train['price_range']==3)].shape[0])


Number of very high priced phones with no camera 
21


**21 Very Highly priced mobile phones have no camera??? Doesn't make sense.....**

**RAM and storage analysis**

In [16]:
temp['ram_gb']=temp['ram']/1024

In [17]:
px.histogram(temp,x='ram_gb',facet_col='price_range',marginal='box')

RAM is the clearest indicator for determining the price of the phone.

In [18]:
px.scatter(temp,x='ram_gb',y='int_memory',color='price_range',facet_col='touch_screen',facet_row='four_g')

Clear distribution is visible here in all the plots and RAM alone might be enough to predict the price_range.

In [19]:
def baseline(x):
    if x>3:
        return 3
    elif x>2 and x<=3:
        return 2
    elif x<=2 and x>1:
        return 1
    elif x<=1:
        return 0


In [20]:
temp['Pred']=temp['ram_gb'].apply(lambda x:baseline(x))

In [21]:
accuracy_score(temp['price_range'],temp['Pred'])

0.7425

The baseline model without any use of machine learning has an accuracy of 0.74 and that is quite good considering the fact that if we constantly predict a single class the accuracy will be 0.25. 

# Predicting the price range 

In [22]:
train.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range', 'px_active', 'screen_area',
       'ram_gb', 'Pred'],
      dtype='object')

The features that generally a user consider before buying a mobile phone are the features needed to predict the price change.
1. battery
2. RAM
3. pixel height-width
4. number of cores
5. memory 
6. camera

In [23]:
train['ram_gb']=train['ram']/1024
test['ram_gb']=test['ram']/1024
train['px_active']=train['px_height']*train['px_width']
test['px_active']=test['px_height']*test['px_width']
train['cam_total']=train['fc']+train['pc']
test['cam_total']=test['fc']+test['pc']

Normalizing the features and creating new features.

In [24]:
X=train[['battery_power','ram_gb','fc','pc','int_memory','n_cores','px_active','touch_screen','four_g']]
y=train['price_range']
X_test=test[['battery_power','ram_gb','fc','pc','int_memory','n_cores','px_active','touch_screen','four_g']]

In [25]:
i=max(X['battery_power'])
X['battery_power']=X['battery_power']/i
X_test['battery_power']=X_test['battery_power']/i

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.25)

In [27]:
forest=RandomForestClassifier(max_depth=8,min_samples_split=15,n_estimators=250)
forest.fit(X_train,y_train)

RandomForestClassifier(max_depth=8, min_samples_split=15, n_estimators=250)

These are the parameters that I reached after running the model a few times and trying to avoid overfitting.

In [28]:
print("The score on the training data ",forest.score(X_train,y_train))

The score on the training data  0.9673333333333334


In [29]:
pred=forest.predict(X_valid)
print('The score on the validation set',accuracy_score(y_valid,pred))

The score on the validation set 0.87


Training the model on the whole training set.

In [30]:
forest=RandomForestClassifier(max_depth=8,min_samples_split=15,n_estimators=250)
forest.fit(X,y)

RandomForestClassifier(max_depth=8, min_samples_split=15, n_estimators=250)

In [31]:
print('Score on the whole training data',forest.score(X,y))

Score on the whole training data 0.9605


Prediction on the test set..

In [32]:
prediction=forest.predict(X_test)